**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Istio Canary

## Hello Istio

Very basic traffic management
https://istio.io/blog/2018/incremental-traffic-management/



In [1]:
kubectl label namespace default istio-injection=enabled

namespace/default labeled


## Create a test pod

In [2]:
kubectl apply -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  name: sleep
  labels:
    app: sleep
spec:
  ports:
  - port: 80
    name: http
  selector:
    app: sleep
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: sleep
spec:
  replicas: 1
  selector:
    matchLabels:
      app: sleep
  template:
    metadata:
      labels:
        app: sleep
    spec:
      containers:
      - name: sleep
        image: pstauffer/curl
        command: ["/bin/sleep", "3650d"]
        imagePullPolicy: IfNotPresent
---
EOF

service/sleep created
deployment.apps/sleep created


In [3]:
export SLEEP_POD=$(kubectl get pod -l app=sleep \
  -o jsonpath={.items..metadata.name})
echo $SLEEP_POD

sleep-56bc8d768-56qrd


In [4]:
kubectl get crd

NAME                                       CREATED AT
authorizationpolicies.security.istio.io    2022-03-29T13:02:44Z
destinationrules.networking.istio.io       2022-03-29T13:02:44Z
envoyfilters.networking.istio.io           2022-03-29T13:02:44Z
gateways.networking.istio.io               2022-03-29T13:02:44Z
istiooperators.install.istio.io            2022-03-29T13:02:44Z
peerauthentications.security.istio.io      2022-03-29T13:02:44Z
proxyconfigs.networking.istio.io           2022-03-29T13:02:44Z
requestauthentications.security.istio.io   2022-03-29T13:02:44Z
serviceentries.networking.istio.io         2022-03-29T13:02:44Z
sidecars.networking.istio.io               2022-03-29T13:02:44Z
telemetries.telemetry.istio.io             2022-03-29T13:02:44Z
virtualservices.networking.istio.io        2022-03-29T13:02:44Z
wasmplugins.extensions.istio.io            2022-03-29T13:02:44Z
workloadentries.networking.istio.io        2022-03-29T13:02:44Z
workloadgroups.networking.istio.io         2022-03

In [5]:
kubectl get destinationrule 

No resources found in default namespace.


In [6]:
kubectl get virtualservice

No resources found in default namespace.


In [7]:
kubectl get deployments

NAME    READY   UP-TO-DATE   AVAILABLE   AGE
sleep   1/1     1            1           46s


In [8]:
kubectl get pods

NAME                    READY   STATUS    RESTARTS   AGE
sleep-56bc8d768-56qrd   2/2     Running   0          51s


In [9]:
kubectl get pod $SLEEP_POD -o yaml | grep --color=always  -E '$|.*image:.*'

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubectl.kubernetes.io/default-container: sleep
    kubectl.kubernetes.io/default-logs-container: sleep
    prometheus.io/path: /stats/prometheus
    prometheus.io/port: "15020"
    prometheus.io/scrape: "true"
    sidecar.istio.io/status: '{"initContainers":["istio-init"],"containers":["istio-proxy"],"volumes":["istio-envoy","istio-data","istio-podinfo","istio-token","istiod-ca-cert"],"imagePullSecrets":null,"revision":"default"}'
  creationTimestamp: "2022-03-29T13:42:52Z"
  generateName: sleep-56bc8d768-
  labels:
    app: sleep
    pod-template-hash: 56bc8d768
    security.istio.io/tlsMode: istio
    service.istio.io/canonical-name: sleep
    service.istio.io/canonical-revision: latest
  name: sleep-56bc8d768-56qrd
  namespace: default
  ownerReferences:
  - apiVersion: apps/v1
    blockOwnerDeletion: true
    controller: true
    kind: ReplicaSet
    name: sleep-56bc8d768
    uid: 51f89ac5-fc55-4a0d-97d4-4b5f901dd350
  resourceV

# Admission Controlers and Sidecars

- Istio works as an [ Admission Controller](https://kubernetes.io/blog/2019/03/21/a-guide-to-kubernetes-admission-controllers/)
- adding a [Sidecar Pattern](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45406.pdf)
- [Kubernetes Patterns](https://www.redhat.com/cms/managed-files/cm-oreilly-kubernetes-patterns-ebook-f19824-201910-en.pdf)
- [Envoy Proxy](https://www.envoyproxy.io/)
- [Patterns with Envoy Proxy](https://blog.christianposta.com/microservices/00-microservices-patterns-with-envoy-proxy-series/)

In [10]:
kubectl exec $SLEEP_POD -- curl  heise.de 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   229  100   229    0     0   1205      0 --:--:-- --:--:-- --:--:--  1198
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>


# Istio Sample Application

In [11]:
ls 

 AccountWithRBAC.ipynb       OPA.ipynb
 AuditLogs.ipynb             OpenPolicyAgent.ipynb
 Calico-Iptables.ipynb       OpenShift
 ClusterOverview.ipynb       Overview.ipynb
 ContainerdCloudbomb.ipynb   PodSecurityPolicyBasic.ipynb
 CrioBomb.ipynb              PodSecurityPolicyPermissive.ipynb
 Harbor.ipynb                PodSecurityPolicyStrict.ipynb
 Hardening-NGinx.ipynb      'Postgres Zalando.ipynb'
 Helm.ipynb                  README.md
 Ingress.ipynb               ServiceAccountToken.ipynb
 IngressKong.ipynb           Unprivileged.ipynb
 IngressKongStart.ipynb      default-rbac.png
 IngressStart.ipynb          defaut-rbac.dot
 Istio-Traffic-Mgmt.ipynb    docker-hypervisor.png
 Istio1.8.2.ipynb            istio-1.13.2
 IstioHack.ipynb             kube-bench.ipynb
 Keycloak.ipynb              kubernetes-client.png
 Kube-Scan-Cloudbomb.ipynb   minikube.dot
 KubeLogin.ipynb             minikube.png
 KubernetesBasics.ipynb      my-namespace-rbac.dot
 License.ipynb               my-name

In [12]:
cd istio-*

In [13]:
pwd

/minikube-host/notebooks/training-kubernetes-security/istio-1.13.2


In [14]:
kubectl apply -f samples/bookinfo/platform/kube/bookinfo.yaml

service/details created
serviceaccount/bookinfo-details created
deployment.apps/details-v1 created
service/ratings created
serviceaccount/bookinfo-ratings created
deployment.apps/ratings-v1 created
service/reviews created
serviceaccount/bookinfo-reviews created
deployment.apps/reviews-v1 created
deployment.apps/reviews-v2 created
deployment.apps/reviews-v3 created
service/productpage created
serviceaccount/bookinfo-productpage created
deployment.apps/productpage-v1 created


In [26]:
kubectl get pods 

NAME                              READY   STATUS    RESTARTS   AGE
details-v1-79f774bdb9-x47gc       2/2     Running   0          68s
productpage-v1-6b746f74dc-hhdph   2/2     Running   0          67s
ratings-v1-b6994bb9-k6m24         2/2     Running   0          68s
reviews-v1-545db77b95-wz4p6       2/2     Running   0          67s
reviews-v2-7bf8c9648f-vxp66       2/2     Running   0          67s
reviews-v3-84779c7bbc-8bqnh       2/2     Running   0          68s
sleep-56bc8d768-56qrd             2/2     Running   0          26m


In [18]:
kubectl get services

NAME          TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
details       ClusterIP   10.102.106.185   <none>        9080/TCP   28s
kubernetes    ClusterIP   10.96.0.1        <none>        443/TCP    4h50m
productpage   ClusterIP   10.105.74.171    <none>        9080/TCP   27s
ratings       ClusterIP   10.104.89.129    <none>        9080/TCP   28s
reviews       ClusterIP   10.107.186.66    <none>        9080/TCP   27s
sleep         ClusterIP   10.101.82.199    <none>        80/TCP     25m


In [28]:
kubectl exec "$(kubectl get pod -l app=ratings -o jsonpath='{.items[0].metadata.name}')" \
  -c ratings -- curl -s productpage:9080/productpage | grep --color=always -E "$|<title>.*</title>"

<!DOCTYPE html>
<html>
  <head>
    <title>Simple Bookstore App</title>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1.0">

<!-- Latest compiled and minified CSS -->
<link rel="stylesheet" href="static/bootstrap/css/bootstrap.min.css">

<!-- Optional theme -->
<link rel="stylesheet" href="static/bootstrap/css/bootstrap-theme.min.css">

  </head>
  <body>
    
    

<nav class="navbar navbar-inverse navbar-static-top">
  <div class="container">
    <div class="navbar-header">
      <a class="navbar-brand" href="#">BookInfo Sample</a>
    </div>
    
    <button type="button" class="btn btn-default navbar-btn navbar-right" data-toggle="modal" href="#login-modal">Sign
      in</button>
    
  </div>
</nav>

<!---
<div class="navbar navbar-inverse navbar-fixed-top">
  <div class="container">
    <div class="navbar-header pull-left">
      <a class="navbar-brand" href="#">Microservices Fabric Bo

In [29]:
cat samples/bookinfo/networking/bookinfo-gateway.yaml

apiVersion: networking.istio.io/v1alpha3
kind: Gateway
metadata:
  name: bookinfo-gateway
spec:
  selector:
    istio: ingressgateway # use istio default controller
  servers:
  - port:
      number: 80
      name: http
      protocol: HTTP
    hosts:
    - "*"
---
apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: bookinfo
spec:
  hosts:
  - "*"
  gateways:
  - bookinfo-gateway
  http:
  - match:
    - uri:
        exact: /productpage
    - uri:
        prefix: /static
    - uri:
        exact: /login
    - uri:
        exact: /logout
    - uri:
        prefix: /api/v1/products
    route:
    - destination:
        host: productpage
        port:
          number: 9080


In [30]:
kubectl apply -f samples/bookinfo/networking/bookinfo-gateway.yaml

gateway.networking.istio.io/bookinfo-gateway created
virtualservice.networking.istio.io/bookinfo created


In [32]:
export PATH=$PATH:$(pwd)/bin

In [33]:
istioctl analyze


✔ No validation issues found when analyzing namespace: default.


In [34]:
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
export SECURE_INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="https")].nodePort}')
export INGRESS_HOST=$(hostname -i)
echo INGRESS_PORT=$INGRESS_PORT
echo SECURE_INGRESS_PORT=$SECURE_INGRESS_PORT
echo INGRESS_HOST=$INGRESS_HOST

INGRESS_PORT=31121
SECURE_INGRESS_PORT=31089
INGRESS_HOST=192.168.49.2


In [35]:
export GATEWAY_URL=$INGRESS_HOST:$INGRESS_PORT
echo "http://$GATEWAY_URL/productpage"

http://192.168.49.2:31121/productpage


In [36]:
kubectl apply -f samples/addons

serviceaccount/grafana created
configmap/grafana created
service/grafana created
deployment.apps/grafana created
configmap/istio-grafana-dashboards created
configmap/istio-services-grafana-dashboards created
deployment.apps/jaeger created
service/tracing created
service/zipkin created
service/jaeger-collector created
serviceaccount/kiali created
configmap/kiali created
clusterrole.rbac.authorization.k8s.io/kiali-viewer created
clusterrole.rbac.authorization.k8s.io/kiali created
clusterrolebinding.rbac.authorization.k8s.io/kiali created
role.rbac.authorization.k8s.io/kiali-controlplane created
rolebinding.rbac.authorization.k8s.io/kiali-controlplane created
service/kiali created
deployment.apps/kiali created
serviceaccount/prometheus created
configmap/prometheus created
clusterrole.rbac.authorization.k8s.io/prometheus created
clusterrolebinding.rbac.authorization.k8s.io/prometheus created
service/prometheus created
deployment.apps/prometheus created


In [42]:
kubectl get pods -n istio-system

NAME                                    READY   STATUS    RESTARTS   AGE
grafana-6c5dc6df7c-5wj2p                1/1     Running   0          39s
istio-egressgateway-66fdd867f4-fqcsp    1/1     Running   0          69m
istio-ingressgateway-77968dbd74-nlkqf   1/1     Running   0          69m
istiod-699b647f8b-ktvqq                 1/1     Running   0          69m
jaeger-9dd685668-99cpx                  1/1     Running   0          38s
kiali-699f98c497-dxhsf                  1/1     Running   0          38s
prometheus-699b7cc575-wqs7s             2/2     Running   0          37s


In [43]:
kubectl get all -n istio-system

NAME                                        READY   STATUS    RESTARTS   AGE
pod/grafana-6c5dc6df7c-5wj2p                1/1     Running   0          44s
pod/istio-egressgateway-66fdd867f4-fqcsp    1/1     Running   0          69m
pod/istio-ingressgateway-77968dbd74-nlkqf   1/1     Running   0          69m
pod/istiod-699b647f8b-ktvqq                 1/1     Running   0          70m
pod/jaeger-9dd685668-99cpx                  1/1     Running   0          43s
pod/kiali-699f98c497-dxhsf                  1/1     Running   0          43s
pod/prometheus-699b7cc575-wqs7s             2/2     Running   0          42s

NAME                           TYPE           CLUSTER-IP       EXTERNAL-IP     PORT(S)                                                                      AGE
service/grafana                ClusterIP      10.104.184.197   <none>          3000/TCP                                                                     44s
service/istio-egressgateway    ClusterIP      10.106.93.246    

In [45]:
istioctl dashboard kiali --address $(hostname -i) | sed s/localhost/$(hostname -i)/&

[1] 86305


In [46]:
fg

http://192.168.49.2:20001/kiali
Failed to open browser; open http://192.168.49.2:20001/kiali in your browser.
istioctl dashboard kiali --address $(hostname -i) | sed s/localhost/$(hostname -i)/



## Cleanup

In [47]:
kubectl delete -f samples/addons -f samples/bookinfo/platform/kube/bookinfo.yaml

serviceaccount "grafana" deleted
configmap "grafana" deleted
service "grafana" deleted
deployment.apps "grafana" deleted
configmap "istio-grafana-dashboards" deleted
configmap "istio-services-grafana-dashboards" deleted
deployment.apps "jaeger" deleted
service "tracing" deleted
service "zipkin" deleted
service "jaeger-collector" deleted
serviceaccount "kiali" deleted
configmap "kiali" deleted
clusterrole.rbac.authorization.k8s.io "kiali-viewer" deleted
clusterrole.rbac.authorization.k8s.io "kiali" deleted
clusterrolebinding.rbac.authorization.k8s.io "kiali" deleted
role.rbac.authorization.k8s.io "kiali-controlplane" deleted
rolebinding.rbac.authorization.k8s.io "kiali-controlplane" deleted
service "kiali" deleted
deployment.apps "kiali" deleted
serviceaccount "prometheus" deleted
configmap "prometheus" deleted
clusterrole.rbac.authorization.k8s.io "prometheus" deleted
clusterrolebinding.rbac.authorization.k8s.io "prometheus" deleted
service "prometheus" deleted
deployment.apps "prometh

In [49]:
kubectl get all

NAME                            READY   STATUS        RESTARTS   AGE
pod/ratings-v1-b6994bb9-k6m24   2/2     Terminating   0          23m
pod/sleep-56bc8d768-56qrd       2/2     Running       0          48m

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP   5h13m
service/sleep        ClusterIP   10.101.82.199   <none>        80/TCP    48m

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/sleep   1/1     1            1           48m

NAME                              DESIRED   CURRENT   READY   AGE
replicaset.apps/sleep-56bc8d768   1         1         1       48m
